# Agent-based Modeling

Agent-based Modeling (ABM) is a simulation method where the autonomous agents interacting with the environment (space) and/or each other by a set of rules.

The most obvious example of ABM is to simulate actions of non-player characters (NPCs) in computer games.

ABM is able to model heterogeneously, i.e. it does not require the environment to be well stirred (as opposed to ODEs), continuous (as opposed to to PDEs), nor need the characteristics of each kind of agents to be identical (as opposed to SSAs).

This makes ABM more flexible to model individual behaviors.
- traffic jam
- infectious disease spread
- molecular interactions

## Elements of ABM

To use `Agents.jl`, we need to define:

- The [**space**](https://juliadynamics.github.io/Agents.jl/stable/api/#Available-spaces-1) where the agents live
- The [**agents**](https://juliadynamics.github.io/Agents.jl/stable/api/#@agent-macro-1) with self-defined properties.
- The **model** to hold the `space`, the `agent`s, and other parameters (called `properties`)
- The stepping function `step!()` to tell how the model evolve.

## Could I do ABM by myself?

Yes, you can define the agents, rules and stepping functions from scratch, but it's more convenient (and perhaps more performant) to use a test package like `Agents.jl`.

## Resources

- [Documentation](https://juliadynamics.github.io/Agents.jl/stable/) of `Agents.jl`.

## Schelling's segregation model

This example is taken from `Agents.jl` [tutorial](https://juliadynamics.github.io/Agents.jl/stable/examples/schelling/).

- Agents : They belong to one of two groups (0 or 1).
- Model : Each position of the grid can be occupied by at most one agent.
- For each step
  - If an agent has at least 3 neighbors belonging to the same group, then it is happy.
  - If an agent is unhappy, it keeps moving to new locations until it is happy.

To define an agent type, we should make a mutable struct derived from `AbstractAgent` with 2 mandatory fields:
- `id::Int` . The identifier number of the agent.
- `pos` . For agents on a 2D grid, the position field should be a tuple of 2 integers.

On top of that, we could define other properties for the agents.

```julia
mutable struct SchellingAgent <: AbstractAgent
    id::Int             # The identifier number of the agent
    pos::NTuple{2, Int} # The x, y location of the agent on a 2D grid
    mood::Bool          # whether the agent is happy in its position. (true = happy)
    group::Int          # The group of the agent, determines mood as it interacts with neighbors
end
```

### Setup

First, we create a 2D space with a Chebyshev metric. This leads to *8 neighboring positions* per position (except at the edges of the grid).

In [ ]:
using Agents

# Creating a space
space = GridSpace((10, 10); periodic = false)

We define the Agent type using the [`@agent`](https://juliadynamics.github.io/Agents.jl/stable/api/#@agent-macro-1) macro. Thus we don't have to setup the mandatory `id` and `pos` fields by ourselves. The relevant fileds are `mood` (whetehr the agent is happy or not) and `group` (which group the agent is on).

In [ ]:
@agent SchellingAgent GridAgent{2} begin
    mood::Bool  # True = happy
    group::Int  # 0 or 1
end

In [ ]:
properties = Dict(:min_to_be_happy => 3) # Parameter for the ABM

schelling = ABM(SchellingAgent, space; properties)
# schelling2 = ABM(SchellingAgent, space; properties = properties, scheduler = Schedulers.by_property(:group),)  # Custom scheduler

We setup the model using an `initialize()` function to make the model easier to reproduce and change its parameter(s).

In [ ]:
using Random # for reproducibility in the RNG

function initialize(; numagents = 320, griddims = (20, 20), min_to_be_happy = 3, seed = 125)
    space = GridSpace(griddims, periodic = false)
    properties = Dict(:min_to_be_happy => min_to_be_happy)
    rng = Random.MersenneTwister(seed)
    model = ABM(
        SchellingAgent, space;
        properties, 
        rng, 
        scheduler = Schedulers.randomly
    )

    # populate the model with agents, adding equal amount of the two types of agents
    # at random positions in the model
    for n in 1:numagents
        agent = SchellingAgent(n, (1, 1), false, n < numagents / 2 ? 1 : 2)
        add_agent_single!(agent, model)  # We don't need to set the starting position. Agents package chooses randomly for us.
    end
    return model
end

Finally, we define a step function `agent_step!()` to determine what happens to each agent. We use some built-in functions: `nearby_agents()` and `move_agent_single!()`.

In [ ]:
function agent_step!(agent, model)
    minhappy = model.min_to_be_happy
    count_neighbors_same_group = 0
    # For each neighbor, get group and compare to current agent's group
    # and increment count_neighbors_same_group as appropriately.
    # Here `nearby_agents` (with default arguments) will provide an iterator
    # over the nearby agents one grid point away, which are at most 8.
    for neighbor in nearby_agents(agent, model)
        if agent.group == neighbor.group
            count_neighbors_same_group += 1
        end
    end
    if count_neighbors_same_group ≥ minhappy
        # The agent is happy
        agent.mood = true
    else
        # Move the agent to a random position
        move_agent_single!(agent, model)  
    end
    return
end

### Evolving the model 

In [ ]:
model = initialize()

The `step!()` function moves the model forward. `run!()` is similar to `step!()` but also collects data along the simulation.

In [ ]:
# Move the model by one iteration
step!(model, agent_step!)

In [ ]:
# Move the model by 3 iterations
step!(model, agent_step!, 3)

### Visualization

In [ ]:
using InteractiveDynamics
using CairoMakie            # Makie with the Cario backend

# Some helper functions to identify agent groups.
groupcolor(a) = a.group == 1 ? :blue : :orange
groupmarker(a) = a.group == 1 ? :circle : :rect

In [ ]:
model = initialize(griddims = (50, 50), numagents = 1800)
figure, _ = abmplot(model; ac = groupcolor, am = groupmarker, as = 10)
figure  # The static figure

In [ ]:
model = initialize(griddims = (50, 50), numagents = 1800)

abmvideo(
    "schelling.mp4", model, agent_step!;
    ac = groupcolor, am = groupmarker, as = 10,
    framerate = 4, frames = 20,
    title = "Schelling's segregation model"
)

We use a helper function to embed MP4 video into the output cell.

In [ ]:
using Base64

function display_mp4(filename)
    display("text/html", string("""<video autoplay controls><source src="data:video/x-m4v;base64,""",
        Base64.base64encode(open(read, filename)),"""" type="video/mp4"></video>"""))
end

In [ ]:
display_mp4("schelling.mp4")

### Data analysis

The `run!()` function rusn simulation and collects data in the `DataFrame` format. The `adata` (aggregated data) keyword extacts information in the DataFrame.

In [ ]:
# aggregated data. fields we want to extract
adata = [:pos, :mood, :group]

model = initialize()
data, _ = run!(model, agent_step!, 5; adata)
data[1:10, :] # print only the first 10 rows

In [ ]:
# adata also accepts functions
x(agent) = agent.pos[1]
model = initialize()
adata = [x, :mood, :group]
data, _ = run!(model, agent_step!, 5; adata)
data[1:10, :]

### Launching an interactive app

See [this section](https://juliadynamics.github.io/Agents.jl/stable/examples/schelling/#Launching-the-interactive-application-1) using `abm_data_exploration()` in the official tutorial.

### Saving the model state

- `AgentsIO.save_checkpoint()`
- `AgentsIO.load_checkpoint()`

In [ ]:
model = initialize(numagents = 200, min_to_be_happy = 5, seed = 42)

run!(model, agent_step!, 40)

figure, _ = abmplot(model; ac = groupcolor, am = groupmarker, as = 10)
# State after 40 steps
figure

In [ ]:
# Save the model state (to be loaded later)
# jld2 as to Julia is "pickle" as to Python
AgentsIO.save_checkpoint("schelling.jld2", model)

In [ ]:
# Load the model
model = AgentsIO.load_checkpoint("schelling.jld2"; scheduler = Schedulers.randomly)

In [ ]:
# Mees with this model by adding more blue agents
for i in 1:100
    agent = SchellingAgent(nextid(model), (1, 1), false, 1)
    add_agent_single!(agent, model)
end

In [ ]:
figure, _ = abmplot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

In [ ]:
step!(model, agent_step!, 40)

figure, _ = abmplot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

And let's try what happens if we add one more group (green) of agents.

In [ ]:
model = AgentsIO.load_checkpoint("schelling.jld2"; scheduler = Schedulers.randomly)

for i in 1:100
    agent = SchellingAgent(nextid(model), (1, 1), false, 3)
    add_agent_single!(agent, model)
end

groupcolor(a) = (:blue, :orange, :green)[a.group]
groupmarker(a) = (:circle, :rect, :cross)[a.group]

figure, _ = abmplot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

In [ ]:
step!(model, agent_step!, 40)

figure, _ = abmplot(model; ac = groupcolor, am = groupmarker, as = 10)
figure

## The spread of SARS-CoV-2 (Graph model)

[Source](https://juliadynamics.github.io/Agents.jl/stable/examples/sir/) from Agents.jl tutorial

Here we add one more category of individuals: those who are infected, but do not know it. Transmission rate for infected and diagnosed individuals is lower than infected and undetected. 

In [ ]:
using Agents, Random, DataFrames, Graphs
using Distributions: Poisson, DiscreteNonParametric
using CairoMakie

### Define the Model

In [ ]:
mutable struct PoorSoul <: AbstractAgent
    id::Int             # Unique agent ID
    pos::Int            # Which city
    days_infected::Int  # number of days since is infected
    status::Symbol      # S/I/R
end

In [ ]:
function make_SIRgraph(;
    Ns,                 # Populations of the cities
    migration_rates,    # Rate of people moving from one city to another
    β_und,              # Transmission rate of infected but undetected
    β_det,              # Transmission rate of infected and detected
    infection_period = 30,
    reinfection_probability = 0.05,
    detection_time = 14,
    death_rate = 0.02,
    Is = [zeros(Int, length(Ns) - 1)..., 1],  # An array for initial number of infected but undetected people per city.
    seed = 2022,
)

    rng = MersenneTwister(seed)

    @assert length(Ns) == length(Is) == length(β_und) == length(β_det) == size(migration_rates, 1) "length of Ns, Is, and B, and number of rows/columns in migration_rates should be the same "
    @assert size(migration_rates, 1) == size(migration_rates, 2) "migration_rates rates should be a square matrix"

    C = length(Ns) # Number of cities
	
    # normalize migration_rates
    migration_rates_sum = sum(migration_rates, dims = 2)
    for c in 1:C
        migration_rates[c, :] ./= migration_rates_sum[c]
    end

    # properties as a NamedTuple
    properties = (;
        Ns,
        Is,
        β_und,
        β_det,
        migration_rates,
        infection_period,
        reinfection_probability,
        detection_time,
        C,
        death_rate
    )
	
	
    space = GraphSpace(complete_digraph(C))
    model = ABM(PoorSoul, space; properties, rng)

    # Add initial susceptible individuals
    for city in 1:C, n in 1:Ns[city]
        ind = add_agent!(city, model, 0, :S)
    end
    # add infected individuals
    for city in 1:C
        inds = ids_in_position(city, model)
        for n in 1:Is[city]
            agent = model[inds[n]]
            agent.status = :I # Infected
            agent.days_infected = 1
        end
    end
    return model
end

In [ ]:
using LinearAlgebra: diagind

function make_SIRgraphParams(;
	C,
    max_travel_rate,
    infection_period = 30,
    reinfection_probability = 0.05,
    detection_time = 14,
    death_rate = 0.02,
    Is = [zeros(Int, C - 1)..., 1],
    seed = 2022,
)
	# For reproducibility
	Random.seed!(seed)
	
	# City population
    Ns = rand(50:5000, C)
	
	# Undetected transmission
    β_und = rand(0.3:0.02:0.6, C)
	
	# Detected transmission (set to 10% of undetected)
    β_det = β_und ./ 10
	
	# Migrate from city i to city j
	# People in small cities tend to migrate to bigger cities
	migration_rates = zeros(C, C)
    for c in 1:C, c2 in 1:C
        migration_rates[c, c2] = (Ns[c] + Ns[c2]) / Ns[c]
    end
	
	# Normalize migration rates
	maxM = maximum(migration_rates)
    migration_rates = (migration_rates .* max_travel_rate) ./ maxM
	
	# Migrate to self = 1
    migration_rates[diagind(migration_rates)] .= 1.0
	
	return (; Ns,
        β_und,
        β_det,
        migration_rates,
        infection_period,
        reinfection_probability,
        detection_time,
        death_rate,
        Is)
end

In [ ]:
SIRgraphparams = make_SIRgraphParams(C = 8, max_travel_rate = 0.01)

In [ ]:
# Stepping function in the SIR Agent-based model
function migrate!(agent, model)
    pid = agent.pos
    d = DiscreteNonParametric(1:(model.C), model.migration_rates[pid, :])
    m = rand(model.rng, d)
    if m ≠ pid
        move_agent!(agent, m, model)
    end
end

function transmit!(agent, model)
    agent.status == :S && return
    rate = if agent.days_infected < model.detection_time
        model.β_und[agent.pos]
    else
        model.β_det[agent.pos]
    end

    d = Poisson(rate)
    n = rand(model.rng, d)
    n == 0 && return

    for contactID in ids_in_position(agent, model)
        contact = model[contactID]
        if contact.status == :S ||
           (contact.status == :R && rand(model.rng) ≤ model.reinfection_probability)
            contact.status = :I
            n -= 1
            n == 0 && return
        end
    end
end

# Count infected days of the agent
update!(agent, model) = agent.status == :I && (agent.days_infected += 1)

function recover_or_die!(agent, model)
    if agent.days_infected ≥ model.infection_period
        if rand(model.rng) ≤ model.death_rate
            kill_agent!(agent, model)
        else
            agent.status = :R
            agent.days_infected = 0
        end
    end
end

function agent_step!(agent::PoorSoul, model)
    migrate!(agent, model)
    transmit!(agent, model)
    update!(agent, model)
    recover_or_die!(agent, model)
end

In [ ]:
model = make_SIRgraph(; SIRgraphparams...)

### Animation

At the moment [abmplot](https://juliadynamics.github.io/Agents.jl/stable/agents_visualizations/#InteractiveDynamics.abmplot) does not plot `GraphSpace`s, but we can still utilize the [ABMObservable](https://juliadynamics.github.io/Agents.jl/stable/agents_visualizations/#InteractiveDynamics.ABMObservable). We do not need to collect data here, only the current status of the model will be used in visualization.

In [ ]:
using InteractiveDynamics
using CairoMakie

abmobs = ABMObservable(model; agent_step!)  # Observable: The quantity that updates dynamically and interactively

In [ ]:
infected_fraction(m, x) = count(m[id].status == :I for id in x) / length(x)
infected_fractions(m) = [infected_fraction(m, ids_in_position(p, m)) for p in positions(m)]

# Connect (lift) model obervable to fracs, color, and the title.
fracs = lift(infected_fractions, abmobs.model)
color = lift(fs -> [cgrad(:inferno)[f] for f in fs], fracs)
title = lift(
    (s, m) -> "step = $(s), infected = $(round(Int, 100infected_fraction(m, allids(m))))%",
    abmobs.s, abmobs.model
)

In [ ]:
fig = Figure(resolution = (600, 400))
ax = Axis(fig[1, 1]; title, xlabel = "City", ylabel = "Population")
barplot!(ax, model.Ns; strokecolor = :black, strokewidth = 1, color)
fig

In [ ]:
record(fig, "covid_evolution.mp4"; framerate = 5) do io
    for j in 1:30
        recordframe!(io)
        Agents.step!(abmobs, 1)
    end
    recordframe!(io)
end

display_mp4("covid_evolution.mp4")

### Data Collection

In [ ]:
# Helper functions
infected(x) = count(i == :I for i in x)
recovered(x) = count(i == :R for i in x)

In [ ]:
model = make_SIRgraph(; SIRgraphparams...)

to_collect = [(:status, f) for f in (infected, recovered, length)]
data, _ = run!(model, agent_step!, 100; adata = to_collect)
data[1:10, :]

In [ ]:
N = sum(model.Ns) # Total initial population
x = data.step
fig = Figure(resolution = (600, 400))
ax = fig[1, 1] = Axis(fig, xlabel = "steps", ylabel = "log10(count)")
li = lines!(ax, x, log10.(data[:, aggname(:status, infected)]), color = :blue)
lr = lines!(ax, x, log10.(data[:, aggname(:status, recovered)]), color = :red)
dead = log10.(N .- data[:, aggname(:status, length)])
ld = lines!(ax, x, dead, color = :green)
fig[1, 2] = Legend(fig, [li, lr, ld], ["infected", "recovered", "dead"])
fig

The exponential growth is clearly visible since the logarithm of the number of infected increases linearly, until everyone is infected.

## Flocking of birds

The flock model illustrates how flocking behavior can emerge when each bird follows three simple rules:

- maintain a minimum distance from other birds to avoid collision
- fly towards the average position of neighbors
- fly in the average direction of neighbors

### Model setup

In [ ]:
using Agents, LinearAlgebra

mutable struct Bird <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}   # Moving in ContinuousSpace
    speed::Float64           # How far the bird travels
    cohere_factor::Float64   # the importance of maintaining the average position of neighbors
    separation::Float64      # the minimum distance a bird must maintain from its neighbors
    separate_factor::Float64 # the importance of maintaining the minimum distance from neighboring birds
    match_factor::Float64    # the importance of matching the average trajectory of neighboring birds
    visual_distance::Float64 # the distance a bird can see and defines a radius of neighboring birds
end

In [ ]:
function initialize_model(;
    n_birds = 100,
    speed = 1.0,
    cohere_factor = 0.25,
    separation = 4.0,
    separate_factor = 0.25,
    match_factor = 0.01,
    visual_distance = 5.0,
    extent = (100, 100),
    spacing = visual_distance / 1.5,
)
    space2d = ContinuousSpace(extent, spacing)
    model = ABM(Bird, space2d, scheduler = Schedulers.randomly)
    for _ in 1:n_birds
        vel = Tuple(rand(model.rng, 2) * 2 .- 1)
        add_agent!(
            model,
            vel,
            speed,
            cohere_factor,
            separation,
            separate_factor,
            match_factor,
            visual_distance,
        )
    end
    return model
end

In [ ]:
function agent_step!(bird, model)
    # Obtain the ids of neighbors within the bird's visual distance
    neighbor_ids = nearby_ids(bird, model, bird.visual_distance)
    N = 0
    match = separate = cohere = (0.0, 0.0)
    # Calculate behaviour properties based on neighbors
    for id in neighbor_ids
        N += 1
        neighbor = model[id].pos
        heading = neighbor .- bird.pos

        # `cohere` computes the average position of neighboring birds
        cohere = cohere .+ heading
        if edistance(bird.pos, neighbor, model) < bird.separation
            # `separate` repels the bird away from neighboring birds
            separate = separate .- heading
        end
        # `match` computes the average trajectory of neighboring birds
        match = match .+ model[id].vel
    end
    N = max(N, 1)
    # Normalise results based on model input and neighbor count
    cohere = cohere ./ N .* bird.cohere_factor
    separate = separate ./ N .* bird.separate_factor
    match = match ./ N .* bird.match_factor
    # Compute velocity based on rules defined above
    bird.vel = (bird.vel .+ cohere .+ separate .+ match) ./ 2
    bird.vel = bird.vel ./ norm(bird.vel)
    # Move bird according to new velocity and speed
    move_agent!(bird, model, bird.speed)
end

### Plotting

In [ ]:
using InteractiveDynamics
using CairoMakie

const bird_polygon = Polygon(Point2f[(-0.5, -0.5), (1, 0), (-0.5, 0.5)])
function bird_marker(b::Bird)
    φ = atan(b.vel[2], b.vel[1]) #+ π/2 + π
    scale(rotate2D(bird_polygon, φ), 2)
end

model = initialize_model()
figure, = abmplot(model; am = bird_marker)
figure

### Animation

In [ ]:
model = initialize_model()

abmvideo(
    "flocking.mp4", model, agent_step!;
    am = bird_marker,
    framerate = 20, 
    frames = 200,
    title = "Flocking"
)

display_mp4("flocking.mp4")

## COVID-19 social distancing model

Source: [Agents.jl model zoo](https://juliadynamics.github.io/AgentsExampleZoo.jl/dev/examples/social_distancing/)

In [ ]:
using Agents
using Random

Let us first create a simple model where balls move around in a continuous space. We need to create agents that comply with `ContinuousSpace`, i.e. they have a pos and vel fields, both of which are tuples of float numbers.

In [ ]:
mutable struct Agent <: AbstractAgent
    id::Int                 # Mandatory Agent identifier
    pos::NTuple{2,Float64}  # Position, required for agents in the ContinuousSpace
    vel::NTuple{2,Float64}  # Moving speeds
    mass::Float64           # Can move or not
end

In [ ]:
function ball_model(; speed = 0.002, seed = 42)
    space2d = ContinuousSpace((1, 1), 0.02)
    model = ABM(Agent, space2d, properties = Dict(:dt => 1.0), rng = MersenneTwister(seed))

    # Add agents to the model
    for ind in 1:500
        pos = Tuple(rand(model.rng, 2))
        vel = sincos(2π * rand(model.rng)) .* speed
        mass = 1.0
        add_agent!(pos, model, vel, mass)
    end
    return model
end

model = ball_model()

In [ ]:
# Agents.move_agent!()
agent_step!(agent, model) = move_agent!(agent, model, model.dt)

In [ ]:
using InteractiveDynamics
using CairoMakie

abmvideo(
    "socialdist1.mp4",
    model,
    agent_step!;
    title = "Ball Model",
    frames = 50,
    spf = 2,
    framerate = 25,
)

display_mp4("socialdist1.mp4")

As you can see the agents move in a straight line in a periodic space without interactions. Let's change that.

### Billiard-like interaction

Using the continuous space API:

- `interacting_pairs()`
- `elastic_collision!()`

And we redefine the stepping function:

In [ ]:
function model_step!(model)
    for (a1, a2) in interacting_pairs(model, 0.012, :nearest)
        elastic_collision!(a1, a2, :mass)
    end
end

In [ ]:
model2 = ball_model()

abmvideo(
    "socialdist2.mp4",
    model2,
    agent_step!,
    model_step!;
    title = "Billiard-like",
    frames = 50,
    spf = 2,
    framerate = 25,
)

display_mp4("socialdist2.mp4")

### Immovable agents

For the following social distancing example, it will become crucial that some agents don't move, and can't be moved (i.e. they stay "isolated"). This is very easy to do with the elastic_collision! function, we only have to make some agents have infinite mass.

In [ ]:
model3 = ball_model()

for id in 1:400
    agent = model3[id]
    agent.mass = Inf
    agent.vel = (0.0, 0.0)
end

abmvideo(
    "socialdist3.mp4",
    model3,
    agent_step!,
    model_step!;
    title = "Billiard-like with stationary agents",
    frames = 50,
    spf = 2,
    framerate = 25,
)

display_mp4("socialdist3.mp4")

### Modeling Virus spread by the SIR model

The agents can be infected with a disease and transfer the disease to other healthy agents around them.

In [ ]:
mutable struct PoorSoul <: AbstractAgent
    id::Int
    pos::NTuple{2,Float64}
    vel::NTuple{2,Float64}
    mass::Float64
    days_infected::Int  # number of days since is infected
    status::Symbol  # :S, :I or :R
    β::Float64
end

 β is the transmission probability, which we choose to make an agent parameter instead of a model parameter. It reflects the level of hygiene of an individual. 

 And the model creation function becomes:

In [ ]:
const steps_per_day = 24 # One tick per hour

function sir_initiation(;
    infection_period = 30 * steps_per_day,
    detection_time = 14 * steps_per_day,
    reinfection_probability = 0.05,
    isolated = 0.0, # in percentage
    interaction_radius = 0.012,
    dt = 1.0,
    speed = 0.002,
    death_rate = 0.044,
    N = 1000,
    initial_infected = 5,
    seed = 42,
    βmin = 0.4,
    βmax = 0.8,
)

    properties = (;
        infection_period,
        reinfection_probability,
        detection_time,
        death_rate,
        interaction_radius,
        dt,
    )
    space = ContinuousSpace((1,1), 0.02)
    model = ABM(PoorSoul, space, properties = Dict(pairs(properties)), rng = MersenneTwister(seed))

    # Add initial individual agents
    for ind in 1:N
        pos = Tuple(rand(model.rng, 2))
        status = ind ≤ N - initial_infected ? :S : :I
        isisolated = ind ≤ isolated * N
        mass = isisolated ? Inf : 1.0
        vel = isisolated ? (0.0, 0.0) : sincos(2π * rand(model.rng)) .* speed

        β = (βmax - βmin) * rand(model.rng) + βmin
        add_agent!(pos, model, vel, mass, 0, status, β)
    end

    return model
end

To visualize this model, we will use black color for the susceptible, red for the infected infected and green for the recovered.

In [ ]:
sir_model = sir_initiation()

sir_colors(a) = a.status == :S ? "#2b2b33" : a.status == :I ? "#bf2642" : "#338c54"

fig, abmstepper = abmplot(sir_model; ac = sir_colors)
fig # display figure

Modify the `model_step!` function to simulate transmission.

In [ ]:
function transmit!(a1, a2, rp)

    # for transmission, only 1 can have the disease (otherwise nothing happens)
    count(a.status == :I for a in (a1, a2)) ≠ 1 && return
    infected, healthy = a1.status == :I ? (a1, a2) : (a2, a1)

    # Lucky and not infected
    rand(model.rng) > infected.β && return

    # Risk of reinfection
   if healthy.status == :R
        rand(model.rng) > rp && return
    end

    # You got virus
    healthy.status = :I
end

function sir_model_step!(model)
    r = model.interaction_radius
    for (a1, a2) in interacting_pairs(model, r, :all)
        transmit!(a1, a2, model.reinfection_probability)
        elastic_collision!(a1, a2, :mass)
    end
end

# Agent-specific functions
function update!(agent) 
    if agent.status == :I
        agent.days_infected += 1
    end
end

function recover_or_die!(agent, model)
    if agent.days_infected ≥ model.infection_period
        if rand(model.rng) ≤ model.death_rate
            kill_agent!(agent, model)
        else
            agent.status = :R
            agent.days_infected = 0
        end
    end
end

function sir_agent_step!(agent, model)
    move_agent!(agent, model, model.dt)
    update!(agent)
    recover_or_die!(agent, model)
end

Running with default parameters.

In [ ]:
sir_model = sir_initiation()

abmvideo(
    "socialdist4.mp4",
    sir_model,
    sir_agent_step!,
    sir_model_step!;
    title = "SIR model",
    frames = 200,
    ac = sir_colors,
    as = 10,
    spf = 2,
    framerate = 20,
)

display_mp4("socialdist4.mp4")

### Analyzing exponential spread

In [ ]:
infected(x) = count(i == :I for i in x)
recovered(x) = count(i == :R for i in x)
# Aggregated data for number of infected and recovered indivisuals
adata = [(:status, infected), (:status, recovered)]

In [ ]:
# Try different parameters
r1, r2 = 0.02, 0.05
β1, β2 = 0.5, 0.1
sir_model1 = sir_initiation(reinfection_probability = r1, βmax = β1)
sir_model2 = sir_initiation(reinfection_probability = r2, βmax = β1)
sir_model3 = sir_initiation(reinfection_probability = r1, βmax = β2)

data1, _ = run!(sir_model1, sir_agent_step!, sir_model_step!, 3000; adata)
data2, _ = run!(sir_model2, sir_agent_step!, sir_model_step!, 3000; adata)
data3, _ = run!(sir_model3, sir_agent_step!, sir_model_step!, 3000; adata)

data1[(end-10):end, :]

In [ ]:
using CairoMakie

figure = Figure()
ax = figure[1, 1] = Axis(figure; ylabel = "Infected", xlabel="Steps")
l1 = lines!(ax, data1[:, dataname((:status, infected))], color = :orange)
l2 = lines!(ax, data2[:, dataname((:status, infected))], color = :blue)
l3 = lines!(ax, data3[:, dataname((:status, infected))], color = :green)
figure[1, 2] = Legend(figure, [l1, l2, l3], ["r=$r1, beta=$β1", "r=$r2, beta=$β1", "r=$r1, beta=$β2"])

figure

### Social distancing

The best way to model social distancing is to make some agents simply not move (which feels like it approximates reality better).

In [ ]:
sir_model = sir_initiation(isolated = 0.85)
abmvideo(
    "socialdist5.mp4",
    sir_model,
    sir_agent_step!,
    sir_model_step!;
    title = "Social Distancing",
    frames = 200,
    spf = 2,
    ac = sir_colors,
    framerate = 20,
)

display_mp4("socialdist5.mp4")

In [ ]:
r4 = 0.02
sir_model4 = sir_initiation(reinfection_probability = r4, βmax = β1, isolated = 0.85)

data4, _ = run!(sir_model4, sir_agent_step!, sir_model_step!, 3000; adata)

l4 = lines!(ax, data4[:, dataname((:status, infected))], color = :red)
figure[1, 2] = Legend(
    figure,
    [l1, l2, l3, l4],
    ["r=$r1, beta=$β1", "r=$r2, beta=$β1", "r=$r1, beta=$β2", "r=$r4, social distancing"],
)
figure

## Zombie Outbreak

An agents on OpenStreetMap space example

From: https://juliadynamics.github.io/Agents.jl/stable/examples/zombies/

In [ ]:
using Agents
using Random

In [ ]:
@agent Zombie OSMAgent begin
    infected::Bool
    speed::Float64
end

Equivalent to

```julia
mutable struct Zombie <: AbstractAgent
    id::Int
    pos::Tuple{Int, Int, Float64}
    infected::Bool
    speed::Float64
end
```

In [ ]:
function initialise(; seed = 1234)
    map_path = OSM.test_map()
    properties = Dict(:dt => 1 / 60)
    model = ABM(
        Zombie,
        OpenStreetMapSpace(map_path);
        properties = properties,
        rng = Random.MersenneTwister(seed)
    )

    for id in 1:100
        start = random_position(model) # At an intersection
        speed = rand(model.rng) * 5.0 + 2.0 # Random speed from 2-7kmph
        human = Zombie(id, start, false, speed)
        add_agent_pos!(human, model)
        OSM.plan_random_route!(human, model; limit = 50) # try 50 times to find a random route
    end
    # We'll add patient zero at a specific (longitude, latitude)
    start = OSM.nearest_road((9.9351811, 51.5328328), model)
    finish = OSM.nearest_node((9.945125635913511, 51.530876112711745), model)

    speed = rand(model.rng) * 5.0 + 2.0 # Random speed from 2-7kmph
    zombie = add_agent!(start, model, true, speed)
    plan_route!(zombie, finish, model)
    # This function call creates & adds an agent, see `add_agent!`
    return model
end

In [ ]:
function agent_step!(agent, model)
    # Each agent will progress along their route
    # Keep track of distance left to move this step, in case the agent reaches its
    # destination early
    distance_left = move_along_route!(agent, model, agent.speed * model.dt)

    if is_stationary(agent, model) && rand(model.rng) < 0.1
        # When stationary, give the agent a 10% chance of going somewhere else
        OSM.plan_random_route!(agent, model; limit = 50)
        # Start on new route, moving the remaining distance
        move_along_route!(agent, model, distance_left)
    end

    if agent.infected
        # Agents will be infected if they get too close (within 10m) to a zombie.
        map(i -> model[i].infected = true, nearby_ids(agent, model, 0.01))
    end
    return
end

In [ ]:
using InteractiveDynamics
using CairoMakie
ac(agent) = agent.infected ? :green : :black
as(agent) = agent.infected ? 10 : 8
model = initialise()

In [ ]:
abmvideo("outbreak.mp4", model, agent_step!;
    title = "Zombie outbreak", 
    framerate = 15, 
    frames = 200, 
    as, ac)

display_mp4("outbreak.mp4")

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()